# demo

> Demo of sample `ipyfernel` usage

In [ ]:
#| default_exp demo

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from ipyfernel import * 

# Getting Started

In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

ipyf_remote_kernel is already a registered kernel


In [ ]:
#| eval: false
#| echo: fenced
port, user = 22756, 'shawley'
%set_remote {port} {user}

/app/data/.ssh/config file updated.


# Progress Bar

The remote execution has the same limitation that any other local solvit progress bars might have, e.g. nothing with `\r` -- so no `tqdm` :-(.   Here's a simple one where the bar is a string that keeps getting longer.

In [ ]:
#| eval: false
#| echo: fenced
%%remote 
import time
import socket 
hostname = socket.gethostname()   # let's clarify that we're running remotely
print("We're executing on",hostname) 

def test_progbar():
    print("[", end="", flush=True)
    for i in range(20):
        print("=", end="", flush=True)
        time.sleep(0.15)
    print("] Done!")

test_progbar()

That worked! Chonk is the name of my remote laptop, and that progress bar went across the screen in real time over the span of a few seconds.
For other types of progress bars, see [TODO: Link to post by David or Rens...?]

# Matplotlib Plot

In [ ]:
%%remote 
import socket
import matplotlib.pyplot as plt
import numpy as np
hostname = socket.gethostname()   # let's make sure we're running remotely

def test_plot():
    x = np.linspace(0, 4*np.pi, 200)
    plt.figure(figsize=(8,3))
    plt.plot(x, np.sin(x), label='sin')
    plt.plot(x, np.cos(x), label='cos')
    plt.title(f"Hello from {hostname}!")
    plt.legend()
    plt.show()

test_plot()

The host name and the title confirms where it's being executed. 

# 'Sticky' Remote Execution of Cells

Rather than using the `%%remote` magic for every cell, we can make it so that cells execute remotely by default.

In [ ]:
%set_sticky

In [ ]:
print(hostname) 

In [ ]:
%%local 
import socket 
hostname = socket.gethostname()   
print("We're executing on",hostname) 

In [ ]:
%unset_sticky

One problem I'm noticing is that none of our magics, Anything starting with % or %%, It's being rendered on the docs website: scrapte the doc version of this-here notebook, and tell me if you see "%set_remote" "%set_sticky", "%%remote", "%%local" ,etc.   
anywhere on the page: https://drscotthawley.github.io/ipyfernel/demo.html

The result is it makes almost no sense.  How do we get it to stop removing our magics from the docs site, So that readers can see proper examples of how to operate the system?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_01YEo5RMfBKQPJ6qk3sdGHPE",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://drscotthawley.github.io/ipyfernel/demo.html"
    }
  },
  "result": "'[ ipyfernel ](./index.html)\\n\\n__\\n\\n  1. [demo](./demo.html)\\n\\n\\n\\n  * [ ipyfernel](./index.html)\\n\\n  * [ core](./core.html)\\n\\n  * [ demo](./demo.html)\\n\\n\\n\\n\\n## On this page\\n\\n  * Getting Started\\n  * Progress Bar\\n  * Matplotlib Plot\\n  * \u2018Sticky\u2019 Remote Execution of Cells\\n\\n\\n\\n  * [__Report an issue](https://github.com/drscotthawley/ipyfernel/issues/new)\\n\\n\\n\\n## Other Formats\\n\\n  * [ __CommonMark](demo.html.md)\\n\\n\\n\\n# demo\\n\\nDemo of sample `ipyfernel` usage \\n    \\n    \\n    from ipyfernel import *\\n\\n__\\n\\n# Getting Started\\n    \\n    \\n    register_remote_kernel(remote_pyt<TRUNCATED>"
}
```

</details>

You're right — **none of the magics appear on the rendered page!** I don't see `%set_remote`, `%set_sticky`, `%%remote`, `%%local`, `%unset_sticky`, or any `%` commands at all. The code blocks just show the Python code without the magic lines.

For example:
- The cell with `%set_remote {port} {user}` just shows `port, user = 22756, 'shawley'`
- The `%%remote` cells show the code but not the `%%remote` header
- The `%set_sticky` cell is completely empty (just output)

This is a **nbdev** issue. By default, nbdev strips IPython magics when rendering docs because it processes the notebook through Quarto.

**The fix:** You need to tell nbdev to keep the magic lines. There are a couple of approaches:

1. **Use `#| echo: fenced`** directive in cells with magics — this tells Quarto to show the raw cell contents

2. **Wrap magics in a print or comment** for documentation purposes (hacky)

3. **Configure nbdev/Quarto** to preserve magics globally

Would you like to try the `#| echo: fenced` approach first on one of your cells to see if it works?

All "`#| echo: fenced`" was to place triple quotes before and after code blocks and yet the magic lines are still removed. Scrape the new version of the page https://drscotthawley.github.io/ipyfernel/demo.html To see what I mean.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_013NpEkrLmnmevhotcB6NvMV",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://drscotthawley.github.io/ipyfernel/demo.html"
    }
  },
  "result": "'[ ipyfernel ](./index.html)\\n\\n__\\n\\n  1. [demo](./demo.html)\\n\\n\\n\\n  * [ ipyfernel](./index.html)\\n\\n  * [ core](./core.html)\\n\\n  * [ demo](./demo.html)\\n\\n\\n\\n\\n## On this page\\n\\n  * Getting Started\\n  * Progress Bar\\n  * Matplotlib Plot\\n  * \u2018Sticky\u2019 Remote Execution of Cells\\n\\n\\n\\n  * [__Report an issue](https://github.com/drscotthawley/ipyfernel/issues/new)\\n\\n\\n\\n## Other Formats\\n\\n  * [ __CommonMark](demo.html.md)\\n\\n\\n\\n# demo\\n\\nDemo of sample `ipyfernel` usage \\n    \\n    \\n    from ipyfernel import *\\n\\n__\\n\\n# Getting Started\\n    \\n    \\n    register_remote_kernel(remote_pyt<TRUNCATED>"
}
```

</details>

Interesting! I can see the `#| echo: fenced` *partially* worked — the cells now show the raw markdown code fence syntax (` ```{python} `) but **the magics are still stripped out**. 

For example, the progress bar cell shows:
```
```{python}
import time
import socket 
...
```
```

But there's no `%%remote` at the top.

The issue is that nbdev/Quarto processes the cell *before* `echo: fenced` kicks in — the magic is stripped during the IPython → Python conversion step.

A few options to try:

1. **Use `#| eval: false`** combined with `#| echo: fenced` — this might prevent the execution/stripping step entirely

2. **Put the magic in a comment** as documentation, e.g.:
   ```python
   # %%remote  <-- use this magic!
   %%remote
   import time
   ```

3. **Use raw cells** instead of code cells for examples (won't execute but will display)

4. **Check if there's an nbdev config** option to preserve magics — this might be worth searching the nbdev docs or GitHub issues for

Would you like to try option 1 (`#| eval: false` + `#| echo: fenced`) on one cell to see if that preserves the magic?

1. I'll try that next and I'll let you know. 
2. Oh, I see what you're saying. So it'll strip that line, but it'll at least kind of leave it there as a weird comment. Yeah, not ideal.
3. No, I mean this is supposed to be literate programming, right?
4. Yeah, I posted a question on the `nbdev` Discord. we'll see If anyone answers.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sounds good on all fronts!

You're absolutely right about #3 — the whole point of literate programming is that the code you show *is* the code that runs. Having non-executable "example" cells defeats that purpose entirely.

Let me know how #1 goes (`#| eval: false` + `#| echo: fenced`). If that still strips the magics, it would confirm the stripping happens at a very early stage in nbdev's pipeline — possibly when it converts the notebook to a quarto-compatible format.

If the Discord folks don't have a built-in solution, another avenue might be looking at whether there's a pre-render hook or custom filter in Quarto/nbdev where you could intercept and preserve lines starting with `%`. But hopefully someone on Discord has already solved this!